In [1]:
import numpy as np
import pandas as pd
from pandas.io.html import read_html
from sklearn.cluster import KMeans
import folium
import requests
from unidecode import unidecode
import requests

In [9]:
# scrab the districts data:

page = 'https://www.citypopulation.de/php/egypt-greatercairo.php'
df = read_html(page, match='Name', header=0) # a list of dataframes
df = df[0]#getting the dataframe frm the list
df.head()

,Name,Native,Status,Governorate,PopulationCensus1996-11-19,PopulationCensus2006-11-11,PopulationEstimate2018-07-01,Unnamed: 7
0,15 Māyū [15th of May City],قسم 15 مايو,Kism (fully urban),Al-Qāhirah,65560,90740,95313,→
1,'Ābidīn,قسم عابدين,Kism (fully urban),Al-Qāhirah,48704,42223,41076,→
2,Ad-Darb al-Aḥmar,قسم الدرب الأحمر,Kism (fully urban),Al-Qāhirah,78375,60488,59578,→
3,Ad-Duqqī,قسم الدقي,Kism (fully urban),Al-Jīzah,...,83430,72304,→
4,'Ain Schams,قسم عين شمس,Kism (fully urban),Al-Qāhirah,469030,525034,625848,→


In [10]:
df.tail()

,Name,Native,Status,Governorate,PopulationCensus1996-11-19,PopulationCensus2006-11-11,PopulationEstimate2018-07-01,Unnamed: 7
70,Shubrā,قسم شبرا,Kism (fully urban),Al-Qāhirah,83753,71118,78128,→
71,Shubrā al-Khaymah 1 [Shubra el-Kheima],قسم أول شبرا الخيمة,Kism (fully urban),Al-Qalyūbyah,416812,461689,492825,→
72,Shubrā al-Khaymah 2 [Shubra el-Kheima],قسم ثان شبرا الخيمة,Kism (fully urban),Al-Qalyūbyah,453963,563880,694922,→
73,Ṭurah [Tura],قسم طره,Kism (fully urban),Al-Qāhirah,67270,94107,234559,→
74,Al-Qāhirah al-Kubrā [Greater Cairo],القاهرة الكبرى,Metropolitan Area,NaN,12600000,15628325,20577113,NaN


In [11]:
# the last row is the summation of the population of all the districts, i will drop it:
df.dropna(inplace=True)

# I will also replace non ascii characters with ascii:
df['Name'] = df.Name.apply(unidecode)
df['Governorate'] = df.Governorate.apply(unidecode)
df.head(10)

,Name,Native,Status,Governorate,PopulationCensus1996-11-19,PopulationCensus2006-11-11,PopulationEstimate2018-07-01,Unnamed: 7
0,15 Mayu [15th of May City],قسم 15 مايو,Kism (fully urban),Al-Qahirah,65560,90740,95313,→
1,'Abidin,قسم عابدين,Kism (fully urban),Al-Qahirah,48704,42223,41076,→
2,Ad-Darb al-Ahmar,قسم الدرب الأحمر,Kism (fully urban),Al-Qahirah,78375,60488,59578,→
3,Ad-Duqqi,قسم الدقي,Kism (fully urban),Al-Jizah,...,83430,72304,→
4,'Ain Schams,قسم عين شمس,Kism (fully urban),Al-Qahirah,469030,525034,625848,→
5,Al-Ahram,قسم الأهرام,Kism (fully urban),Al-Jizah,...,...,672064,→
6,Al-'Ajuzah,قسم العجوزة,Kism (fully urban),Al-Jizah,...,258511,283876,→
7,Al-Amiriiah,قسم الاميريه,Kism (fully urban),Al-Qahirah,164211,165407,155402,→
8,Al-Azbakiyah [Azbakeya],قسم الأزبكية,Kism (fully urban),Al-Qahirah,30375,28033,20132,→
9,Al-Badrashayn [Badrshein],مركز البدرشين,Markaz,Al-Jizah,285858,383610,551894,→


In [12]:
df['Governorate'].unique()

array(['Al-Qahirah', 'Al-Jizah', 'Al-Qalyubyah', 'Ash-Sharqiyah'],
      dtype=object)

In [14]:
# convert governorate names to the formal names: AlQahira: Cairo - Jizah: Giza
for i, gov in enumerate(df.Governorate.values):
    if gov == 'Al-Qahirah':
        df.at[i, 'Governorate'] = 'Cairo'
        
    elif gov == 'Al-Jizah':
        df.at[i, 'Governorate'] = 'Giza'
        
df.head()

,Name,Native,Status,Governorate,PopulationCensus1996-11-19,PopulationCensus2006-11-11,PopulationEstimate2018-07-01,Unnamed: 7
0,15 Mayu [15th of May City],قسم 15 مايو,Kism (fully urban),Cairo,65560,90740,95313,→
1,'Abidin,قسم عابدين,Kism (fully urban),Cairo,48704,42223,41076,→
2,Ad-Darb al-Ahmar,قسم الدرب الأحمر,Kism (fully urban),Cairo,78375,60488,59578,→
3,Ad-Duqqi,قسم الدقي,Kism (fully urban),Giza,...,83430,72304,→
4,'Ain Schams,قسم عين شمس,Kism (fully urban),Cairo,469030,525034,625848,→


In [15]:
# The [] brackets and the text inside is a repetition, they should be removed. 
# Also still there is a ' at the start of some district names that should be removed. it is already omitted in the second repetition

d = df[['Name', 'Governorate', 'PopulationEstimate2018-07-01']]
names = d['Name'].str.split('[')

for i, x in enumerate(names):  
    if len(x) == 2:
        names[i] = x[1].lstrip('[').rstrip(']')
    else:
        names[i] = x[0].lstrip('[\' ').rstrip(']')

d['Name'] = names
d.head(10)

/home/ingramai/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


,Name,Governorate,PopulationEstimate2018-07-01
0,15th of May City,Cairo,95313
1,Abidin,Cairo,41076
2,Ad-Darb al-Ahmar,Cairo,59578
3,Ad-Duqqi,Giza,72304
4,Ain Schams,Cairo,625848
5,Al-Ahram,Giza,672064
6,Al-'Ajuzah,Giza,283876
7,Al-Amiriiah,Cairo,155402
8,Azbakeya,Cairo,20132
9,Badrshein,Giza,551894


In [16]:
# select columns of interest and make a full address column to use for getting location data
d['Address'] = d['Name'].str.cat(df['Governorate'], sep=',')
d = d[['Address', 'PopulationEstimate2018-07-01']]
d.head()

/home/ingramai/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,Address,PopulationEstimate2018-07-01
0,"15th of May City,Cairo",95313
1,"Abidin,Cairo",41076
2,"Ad-Darb al-Ahmar,Cairo",59578
3,"Ad-Duqqi,Giza",72304
4,"Ain Schams,Cairo",625848


In [ ]:
# loop on the districts to get their location data, and determine those which are not recognized by the geocoding engine if any
latlong = {}
for add in d.Address.values:
    
    url = 'https://us1.locationiq.com/v1/search.php?key=1463232716cb66&q={}&countrycodes=EG&format=json'.format(add)
    results = requests.get(url)    
    try:
        latlong[add] = [results.json()[0]['lat'], results.json()[0]['lon']]
    except:
        latlong[add] = ['NaN', 'NaN']


In [19]:
latlongg = pd.DataFrame.from_dict(latlong, 'index', columns=['Latitude', 'Longitude']).reset_index()
latlongg.head(10)

,index,Latitude,Longitude
0,"15th of May City,Cairo",29.8320418,31.3647679
1,"Abidin,Cairo",30.50473,31.25247
2,"Ad-Darb al-Ahmar,Cairo",30.043259,31.257908
3,"Ad-Duqqi,Giza",NaN,NaN
4,"Ain Schams,Cairo",30.128183,31.333451
5,"Ahram,Giza",30.0030274,31.1909308
6,"'Ajuzah,Giza",30.05061,31.21091
7,"Amiriiah,Cairo",30.06263,31.24967
8,"Azbakeya,Cairo",30.0538891,31.2454725
9,"Badrshein,Giza",NaN,NaN


In [20]:
latlongg[latlongg['Latitude'] == 'NaN']['index']

3        Ad-Duqqi,Giza
9       Badrshein,Giza
32    Ash-Shuruq,Cairo
33      As-Sahil,Cairo
37     At-Talbiah,Giza
39          Ossim,Giza
Name: index, dtype: object

In [21]:
for gov in df['Governorate'].unique():
    print('*****', gov, '*****', d[df['Governorate'] == gov].describe())

***** Cairo *****        PopulationEstimate2018-07-01
count                     46.000000
mean                  211246.173913
std                   201763.779003
min                    10760.000000
25%                    73255.250000
50%                   138594.500000
75%                   272100.000000
max                   813529.000000
***** Giza *****        PopulationEstimate2018-07-01
count                     17.000000
mean                  408616.647059
std                   247961.531693
min                    72304.000000
25%                   200170.000000
50%                   377547.000000
75%                   551894.000000
max                   978626.000000
***** Al-Qalyubyah *****        PopulationEstimate2018-07-01
count                      9.000000
mean                  409594.777778
std                   227184.236397
min                    78185.000000
25%                   149737.000000
50%                   492825.000000
75%                   570192.000000
max 

In [35]:
# I will try removing the letters that are not needed for naming (this is a property for arabic names like: AL - Ash .. etc)
for x in ['Al-', 'Ad-', 'Ash-', 'At-']:
    latlongg['index'] = latlongg['index'].str.replace(x, '')
    
latlongg[latlongg['Latitude'] == 'NaN']['index']

3         Duqqi,Giza
9     Badrshein,Giza
32      Shuruq,Cairo
33    As-Sahil,Cairo
37      Talbiah,Giza
39        Ossim,Giza
67      Talbiah,Giza
Name: index, dtype: object

In [40]:
#lets try to geth the location data of these districts again

for add in latlongg[latlongg['Latitude'] == 'NaN']['index'].values:
    print(add)
    url = 'https://us1.locationiq.com/v1/search.php?key=1463232716cb66&q={}&countrycodes=EG&format=json'.format(add)
    results = requests.get(url)    
    try:
        latlongg[latlongg['index'] == add]['Latitude'] = results.json()[0]['lat']
        latlongg[latlongg['index'] == add]['Longitude'] = results.json()[0]['lon']
    except:
        latlongg[latlongg['index'] == add]['Latitude'] = 'NaN'
        latlongg[latlongg['index'] == add]['Longitude'] = 'NaN'
        

Duqqi,Giza


/home/ingramai/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/ingramai/.local/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Badrshein,Giza


/home/ingramai/.local/lib/python3.6/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/home/ingramai/.local/lib/python3.6/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


Shuruq,Cairo
As-Sahil,Cairo
Talbiah,Giza
Ossim,Giza
Talbiah,Giza


In [41]:
latlongg[latlongg['Latitude'] == 'NaN']['index']

3         Duqqi,Giza
9     Badrshein,Giza
32      Shuruq,Cairo
33    As-Sahil,Cairo
37      Talbiah,Giza
39        Ossim,Giza
67      Talbiah,Giza
Name: index, dtype: object

In [45]:
# This did not work also, so i will just discard these districts as they are not in the database of the geocoder

# I will combine the 2 data frames into one
dd = latlongg.join(d.set_index('Address'), 'index')
dd.dropna()
dd.head()

,index,Latitude,Longitude,PopulationEstimate2018-07-01
0,"15th of May City,Cairo",29.8320418,31.3647679,95313
1,"Abidin,Cairo",30.50473,31.25247,41076
2,"Darb al-Ahmar,Cairo",30.043259,31.257908,59578
3,"Duqqi,Giza",NaN,NaN,72304
4,"Ain Schams,Cairo",30.128183,31.333451,625848


In [50]:
len(dd['index'].unique())

65